In [1]:
#from PyQt5.QtWidgets import QMainWindow, QApplication, QPushButton, QTextEdit, QFileDialog, QWidget
from PyQt5 import uic, QtWidgets
import sys
from PyQt5.QtWidgets import *
from PyQt5.QtGui import *
from PyQt5 import *
import sys
from PyQt5 import QtCore, QtGui, QtWidgets

In [2]:
import argparse
import os
import sys
from pathlib import Path

import torch
import torch.backends.cudnn as cudnn

#FILE = Path(__file__).resolve()
ROOT = os.getcwd()  # YOLOv5 root directory
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))  # add ROOT to PATH
ROOT = Path(os.path.relpath(ROOT, Path.cwd()))  # relative

from models.common import DetectMultiBackend
from utils.datasets import IMG_FORMATS, VID_FORMATS, LoadImages, LoadStreams
from utils.general import (LOGGER, check_file, check_img_size, check_imshow, check_requirements, colorstr, cv2,
                           increment_path, non_max_suppression, print_args, scale_coords, strip_optimizer, xyxy2xywh)
from utils.plots import Annotator, colors, save_one_box
from utils.torch_utils import select_device, time_sync


@torch.no_grad()
def run(weights="" ,  # model.pt path(s)
        source="",  # file/dir/URL/glob, 0 for webcam
        data='data/coco128.yaml',  # dataset.yaml path
        imgsz=(640, 640),  # inference size (height, width)
        conf_thres=0.25,  # confidence threshold
        iou_thres=0.45,  # NMS IOU threshold
        max_det=1000,  # maximum detections per image
        device='',  # cuda device, i.e. 0 or 0,1,2,3 or cpu
        view_img=False,  # show results
        save_txt=False,  # save results to *.txt
        save_conf=False,  # save confidences in --save-txt labels
        save_crop=False,  # save cropped prediction boxes
        nosave=False,  # do not save images/videos
        classes=None,  # filter by class: --class 0, or --class 0 2 3
        agnostic_nms=False,  # class-agnostic NMS
        augment=False,  # augmented inference
        visualize=False,  # visualize features
        update=False,  # update all models
        project=ROOT / 'runs/detect',  # save results to project/name
        name='exp',  # save results to project/name
        exist_ok=False,  # existing project/name ok, do not increment
        line_thickness=3,  # bounding box thickness (pixels)
        hide_labels=False,  # hide labels
        hide_conf=False,  # hide confidences
        half=False,  # use FP16 half-precision inference
        dnn=False,  # use OpenCV DNN for ONNX inference
        ):
    source = str(source)
    save_img = not nosave and not source.endswith('.txt')  # save inference images
    is_file = Path(source).suffix[1:] in (IMG_FORMATS + VID_FORMATS)
    is_url = source.lower().startswith(('rtsp://', 'rtmp://', 'http://', 'https://'))
    webcam = source.isnumeric() or source.endswith('.txt') or (is_url and not is_file)
    if is_url and is_file:
        source = check_file(source)  # download

    # Directories
    save_dir = increment_path(Path(project) / name, exist_ok=exist_ok)  # increment run
    (save_dir / 'labels' if save_txt else save_dir).mkdir(parents=True, exist_ok=True)  # make dir

    # Load model
    device = select_device(device)
    model = DetectMultiBackend(weights, device=device, dnn=dnn, data=data, fp16=half)
    stride, names, pt = model.stride, model.names, model.pt
    imgsz = check_img_size(imgsz, s=stride)  # check image size

    # Dataloader
    if webcam:
        view_img = check_imshow()
        cudnn.benchmark = True  # set True to speed up constant image size inference
        dataset = LoadStreams(source, img_size=imgsz, stride=stride, auto=pt)
        bs = len(dataset)  # batch_size
    else:
        dataset = LoadImages(source, img_size=imgsz, stride=stride, auto=pt)
        bs = 1  # batch_size
    vid_path, vid_writer = [None] * bs, [None] * bs

    # Run inference
    model.warmup(imgsz=(1 if pt else bs, 3, *imgsz))  # warmup
    dt, seen = [0.0, 0.0, 0.0], 0
    for path, im, im0s, vid_cap, s in dataset:
        t1 = time_sync()
        im = torch.from_numpy(im).to(device)
        im = im.half() if model.fp16 else im.float()  # uint8 to fp16/32
        im /= 255  # 0 - 255 to 0.0 - 1.0
        if len(im.shape) == 3:
            im = im[None]  # expand for batch dim
        t2 = time_sync()
        dt[0] += t2 - t1

        # Inference
        visualize = increment_path(save_dir / Path(path).stem, mkdir=True) if visualize else False
        pred = model(im, augment=augment, visualize=visualize)
        t3 = time_sync()
        dt[1] += t3 - t2

        # NMS
        pred = non_max_suppression(pred, conf_thres, iou_thres, classes, agnostic_nms, max_det=max_det)
        dt[2] += time_sync() - t3

        # Second-stage classifier (optional)
        # pred = utils.general.apply_classifier(pred, classifier_model, im, im0s)

        # Process predictions
        for i, det in enumerate(pred):  # per image
            seen += 1
            if webcam:  # batch_size >= 1
                p, im0, frame = path[i], im0s[i].copy(), dataset.count
                s += f'{i}: '
            else:
                p, im0, frame = path, im0s.copy(), getattr(dataset, 'frame', 0)

            p = Path(p)  # to Path
            save_path = str(save_dir / p.name)  # im.jpg
            txt_path = str(save_dir / 'labels' / p.stem) + ('' if dataset.mode == 'image' else f'_{frame}')  # im.txt
            s += '%gx%g ' % im.shape[2:]  # print string
            gn = torch.tensor(im0.shape)[[1, 0, 1, 0]]  # normalization gain whwh
            imc = im0.copy() if save_crop else im0  # for save_crop
            annotator = Annotator(im0, line_width=line_thickness, example=str(names))
            if len(det):
                # Rescale boxes from img_size to im0 size
                det[:, :4] = scale_coords(im.shape[2:], det[:, :4], im0.shape).round()

                # Print results
                for c in det[:, -1].unique():
                    n = (det[:, -1] == c).sum()  # detections per class
                    s += f"{n} {names[int(c)]}{'s' * (n > 1)}, "  # add to string

                # Write results
                for *xyxy, conf, cls in reversed(det):
                    if save_txt:  # Write to file
                        xywh = (xyxy2xywh(torch.tensor(xyxy).view(1, 4)) / gn).view(-1).tolist()  # normalized xywh
                        line = (cls, *xywh, conf) if save_conf else (cls, *xywh)  # label format
                        with open(txt_path + '.txt', 'a') as f:
                            f.write(('%g ' * len(line)).rstrip() % line + '\n')

                    if save_img or save_crop or view_img:  # Add bbox to image
                        c = int(cls)  # integer class
                        label = None if hide_labels else (names[c] if hide_conf else f'{names[c]} {conf:.2f}')
                        annotator.box_label(xyxy, label, color=colors(c, True))
                        if save_crop:
                            save_one_box(xyxy, imc, file=save_dir / 'crops' / names[c] / f'{p.stem}.jpg', BGR=True)

            # Stream results
            im0 = annotator.result()
            if view_img:
                cv2.imshow(str(p), im0)
                cv2.waitKey(1)  # 1 millisecond

            # Save results (image with detections)
            if save_img:
                if dataset.mode == 'image':
                    cv2.imwrite(save_path, im0)
                else:  # 'video' or 'stream'
                    if vid_path[i] != save_path:  # new video
                        vid_path[i] = save_path
                        if isinstance(vid_writer[i], cv2.VideoWriter):
                            vid_writer[i].release()  # release previous video writer
                        if vid_cap:  # video
                            fps = vid_cap.get(cv2.CAP_PROP_FPS)
                            w = int(vid_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
                            h = int(vid_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
                        else:  # stream
                            fps, w, h = 25, im0.shape[1], im0.shape[0]
                        save_path = str(Path(save_path).with_suffix('.mp4'))  # force *.mp4 suffix on results videos
                        vid_writer[i] = cv2.VideoWriter(save_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))
                    vid_writer[i].write(im0)
        # Print time (inference-only)
        LOGGER.info(f'{s}Done. ({t3 - t2:.3f}s)')
        
        
    # Print results
    t = tuple(x / seen * 1E3 for x in dt)  # speeds per image
    LOGGER.info(f'Speed: %.1fms pre-process, %.1fms inference, %.1fms NMS per image at shape {(1, 3, *imgsz)}' % t)
    if save_txt or save_img:
        s = f"\n{len(list(save_dir.glob('labels/*.txt')))} labels saved to {save_dir / 'labels'}" if save_txt else ''
        LOGGER.info(f"Results saved to {colorstr('bold', save_dir)}{s}")
    if update:
        strip_optimizer(weights)  # update model (to fix SourceChangeWarning)

In [3]:
import requests
from xml.etree import ElementTree
import socket
ip = socket.gethostbyname(socket.gethostname())
with requests.Session() as session:
    query = {
    "userName":"yangyoujian","password":"Huawei@2021"
    }
    session.get('https://186.1.16.32:18531/loginInfo/login/v1.0',verify=False)
    response = session.post('https://186.1.16.32:18531/loginInfo/login/v1.0', json  = query,verify=False)
    #CameraCode = "02010533752047770101#da30d0eb264e47968184273537e16acf"
    #CameraCode = "02010533752047770101#da30d0eb264e47968184273537e16acf"
    #CameraCode = "09734697859902870101#cfe82178d732466dae0c21e56c3e8025"
    if response.json()['resultCode'] == 0:
        print("login sucess")
    else:
        print("login failed")

class Ui_Form(object):
    def __init__(self, Form,dict):
        CameraCode = "02010533752047770101#da30d0eb264e47968184273537e16acf"
        query = {
        "cameraCode": CameraCode,
        "mediaURLParam": {
        "serviceType": 1,
        "streamType": 2,
        "clientType" : 1,
        "protocolType": 2,
        }
        } 
        response = session.post('https://186.1.16.32:18531/video/rtspurl/v1.0', json = query, verify=False)
        if response.json()['resultCode'] == 0:
            print("Stream begun")
        else:
            print("Stream failed")
        print(response.content)
        suhail = response.json()['rtspURL']
        print(suhail)
        run(weights="C:\\Users\\OpenLap\\Desktop\\yolov5\\model\\UAV\\UAV.pt" , source=suhail)
        Form.setObjectName("Form")
        Form.resize(622, 423)
        self.label = QtWidgets.QLabel(Form)
        self.label.setGeometry(QtCore.QRect(110, 60, 161, 20))
        self.label.setAutoFillBackground(True)
        self.label.setObjectName("label")
        self.pushButton = QtWidgets.QPushButton(Form)
        self.pushButton.setGeometry(QtCore.QRect(360, 50, 75, 41))
        self.pushButton.setObjectName("pushButton")
        self.tableWidget = QtWidgets.QTableWidget(Form)
        self.tableWidget.setGeometry(QtCore.QRect(10, 160, 591, 192))
        self.tableWidget.setObjectName("tableWidget")
        self.tableWidget.setColumnCount(20)
        self.tableWidget.setRowCount(0)
        item = QtWidgets.QTableWidgetItem()
        self.tableWidget.setHorizontalHeaderItem(0, item)
        item = QtWidgets.QTableWidgetItem()
        self.tableWidget.setHorizontalHeaderItem(1, item)
        item = QtWidgets.QTableWidgetItem()
        self.tableWidget.setHorizontalHeaderItem(2, item)
        item = QtWidgets.QTableWidgetItem()
        self.tableWidget.setHorizontalHeaderItem(3, item)
        item = QtWidgets.QTableWidgetItem()
        self.tableWidget.setHorizontalHeaderItem(4, item)
        item = QtWidgets.QTableWidgetItem()
        self.tableWidget.setHorizontalHeaderItem(5, item)
        item = QtWidgets.QTableWidgetItem()
        self.tableWidget.setHorizontalHeaderItem(6, item)
        item = QtWidgets.QTableWidgetItem()
        self.tableWidget.setHorizontalHeaderItem(7, item)
        item = QtWidgets.QTableWidgetItem()
        self.tableWidget.setHorizontalHeaderItem(8, item)
        item = QtWidgets.QTableWidgetItem()
        self.tableWidget.setHorizontalHeaderItem(9, item)
        item = QtWidgets.QTableWidgetItem()
        self.tableWidget.setHorizontalHeaderItem(10, item)
        item = QtWidgets.QTableWidgetItem()
        self.tableWidget.setHorizontalHeaderItem(11, item)
        item = QtWidgets.QTableWidgetItem()
        self.tableWidget.setHorizontalHeaderItem(12, item)
        item = QtWidgets.QTableWidgetItem()
        self.tableWidget.setHorizontalHeaderItem(13, item)
        item = QtWidgets.QTableWidgetItem()
        self.tableWidget.setHorizontalHeaderItem(14, item)
        item = QtWidgets.QTableWidgetItem()
        self.tableWidget.setHorizontalHeaderItem(15, item)
        item = QtWidgets.QTableWidgetItem()
        self.tableWidget.setHorizontalHeaderItem(16, item)
        item = QtWidgets.QTableWidgetItem()
        self.tableWidget.setHorizontalHeaderItem(17, item)
        item = QtWidgets.QTableWidgetItem()
        self.tableWidget.setHorizontalHeaderItem(18, item)
        item = QtWidgets.QTableWidgetItem()
        self.tableWidget.setHorizontalHeaderItem(19, item)
        self.tableWidget.clicked.connect(self.double_click)
        self.pushButton.clicked.connect(self.click1)
        print(self.tableWidget)
        self.retranslateUi(Form)
        self.load_data(dict)
        QtCore.QMetaObject.connectSlotsByName(Form)
        print("hii")

    def double_click(self):
        print("zzzzzzzzzzzzzzzzzzzzzzzzzz")
        for item in self.tableWidget.selectedItems() :
            self.label.setText(item.text())
    def click1(self):
        response = session.post('https://186.1.16.32:18531/video/rtspurl/v1.0', json = query, verify=False)
        if response.json()['resultCode'] == 0:
            print("Stream begun")
        else:
            print("Stream failed")
        print(response.content)
    def retranslateUi(self, Form):
        _translate = QtCore.QCoreApplication.translate
        Form.setWindowTitle(_translate("Form", "Form"))
        self.label.setText(_translate("Form", " Name of Camera"))
        self.pushButton.setText(_translate("Form", "Camera"))
        item = self.tableWidget.horizontalHeaderItem(0)
        item.setText(_translate("Form", "code"))
        item = self.tableWidget.horizontalHeaderItem(1)
        item.setText(_translate("Form", "Name"))
        item = self.tableWidget.horizontalHeaderItem(2)
        item.setText(_translate("Form", "Device Group Code"))
        item = self.tableWidget.horizontalHeaderItem(3)
        item.setText(_translate("Form", "Parent Code"))
        item = self.tableWidget.horizontalHeaderItem(4)
        item.setText(_translate("Form", "Domain Code"))
        item = self.tableWidget.horizontalHeaderItem(5)
        item.setText(_translate("Form", "Device model type"))
        item = self.tableWidget.horizontalHeaderItem(6)
        item.setText(_translate("Form", "vendor type"))
        item = self.tableWidget.horizontalHeaderItem(7)
        item.setText(_translate("Form", "Device form type"))
        item = self.tableWidget.horizontalHeaderItem(8)
        item.setText(_translate("Form", "Type"))
        item = self.tableWidget.horizontalHeaderItem(9)
        item.setText(_translate("Form", "Camera location"))
        item = self.tableWidget.horizontalHeaderItem(10)
        item.setText(_translate("Form", "Camera status"))
        item = self.tableWidget.horizontalHeaderItem(11)
        item.setText(_translate("Form", "Status"))
        item = self.tableWidget.horizontalHeaderItem(12)
        item.setText(_translate("Form", "Net type"))
        item = self.tableWidget.horizontalHeaderItem(13)
        item.setText(_translate("Form", "Support Intelligent"))
        item = self.tableWidget.horizontalHeaderItem(14)
        item.setText(_translate("Form", "Enable voide"))
        item = self.tableWidget.horizontalHeaderItem(15)
        item.setText(_translate("Form", "nvr Code"))
        item = self.tableWidget.horizontalHeaderItem(16)
        item.setText(_translate("Form", "Device create time"))
        item = self.tableWidget.horizontalHeaderItem(17)
        item.setText(_translate("Form", "Ex Domain "))
        item = self.tableWidget.horizontalHeaderItem(18)
        item.setText(_translate("Form", "Device IP"))
        item = self.tableWidget.horizontalHeaderItem(19)
        item.setText(_translate("Form", "reserve"))
        print("hi2")
    def load_data(self,dict):
        col = 0
        row = 0
        self.tableWidget.setRowCount(len(dict))
        for i in range(len(dict)) :
            col = 0
            for j in dict[i] :
                self.tableWidget.setItem(row,col,QtWidgets.QTableWidgetItem(str(dict[i][j])))
                col+=1

            row+=1

#if _name_ == "_main_":
#    app = QtWidgets.QApplication(sys.argv)
    

    
   
    #sys.exit(app.exec_())
    
class AnotherWindow(QFrame):
    def __init__(self):
        super().__init__()
        self.w = None
        win = uic.loadUi("C:\\Users\\OpenLap\\Downloads\\untitled4.ui",self)
        #self.label = QLabel("Another Window")
        self.pushButton_2.clicked.connect(self.show_new_window2)
    def open_file(self):
        path = QFileDialog.getOpenFileName(self, 'Open a file', '',
                                        'All Files (*.*)')
        if path != ('', ''):
            print(path[0])
            
    def show_new_window2(self, checked):
        query = {"deviceType":2,"fromIndex":1,"toIndex":1000}
        response = session.get('https://186.1.16.32:18531/device/deviceList/v1.0', params = query, verify=False)
        if response.json()['resultCode'] == 0:
            print("camera list sucess")
        else:
            print("camera list failed")
        #print(response.json()['cameraBriefInfos'])
        dict = {}
        dict = response.json()['cameraBriefInfos']['cameraBriefInfoList']['cameraBriefInfo']
        #print(dict)
        if self.w is None:
            self.Form = QtWidgets.QWidget()
            ui  = Ui_Form(self.Form,dict)
        self.Form.show()    
        
        #Form.show()        
class Appdemo(QMainWindow) : 
    def __init__(self) : 
        self.w = None
        super().__init__()    
        win = uic.loadUi("C:\\Users\\OpenLap\\Downloads\\untitled3.ui",self)
        self.pushButton.clicked.connect(self.show_new_window)
        self.show()
    def show_new_window(self, checked):
        if self.w is None:
            self.w = AnotherWindow()
        self.w.show()
    
'''

if __name__ == '__main__' :
    app = QApplication(sys.argv) 
    demo = Appdemo()
    #demo.show()
    try : 
        sys.exit(app.exec_())
    except SystemExit : 
        print("Closing Window")
        '''
app = QtWidgets.QApplication(sys.argv)
window = Appdemo()
sys.exit(app.exec_())


C:\Users\OpenLap\anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host '186.1.16.32'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\OpenLap\anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host '186.1.16.32'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


login sucess


C:\Users\OpenLap\anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host '186.1.16.32'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\OpenLap\anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host '186.1.16.32'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
YOLOv5  v6.1-74-gcf4f3c3 torch 1.11.0+cpu CPU



camera list sucess
Stream begun
b'{"resultCode":0,"rtspURL":"rtsp://186.1.16.32:1161/02010533752047770101?DstCode=01&ServiceType=1&ClientType=1&StreamID=2&SrcTP=2&DstTP=2&SrcPP=1&DstPP=1&MediaTransMode=0&BroadcastType=0&SV=1&Token=xaIND5k1K9WdKdE8qLFtlePMpqjIRTSiMxD4EBzDiMs=&DomainCode=da30d0eb264e47968184273537e16acf&UserId=12&"}'
rtsp://186.1.16.32:1161/02010533752047770101?DstCode=01&ServiceType=1&ClientType=1&StreamID=2&SrcTP=2&DstTP=2&SrcPP=1&DstPP=1&MediaTransMode=0&BroadcastType=0&SV=1&Token=xaIND5k1K9WdKdE8qLFtlePMpqjIRTSiMxD4EBzDiMs=&DomainCode=da30d0eb264e47968184273537e16acf&UserId=12&


Fusing layers... 
YOLOv5m summary: 290 layers, 20861016 parameters, 0 gradients, 48.0 GFLOPs
1/1: rtsp://186.1.16.32:1161/02010533752047770101?DstCode=01&ServiceType=1&ClientType=1&StreamID=2&SrcTP=2&DstTP=2&SrcPP=1&DstPP=1&MediaTransMode=0&BroadcastType=0&SV=1&Token=xaIND5k1K9WdKdE8qLFtlePMpqjIRTSiMxD4EBzDiMs=&DomainCode=da30d0eb264e47968184273537e16acf&UserId=12&...  Success (inf frames 720x576 at 25.00 FPS)

0: 512x640 1 Drone, Done. (0.243s)
0: 512x640 1 Drone, Done. (0.209s)
0: 512x640 1 Drone, Done. (0.211s)
0: 512x640 Done. (0.212s)
0: 512x640 Done. (0.224s)
0: 512x640 1 Drone, Done. (0.215s)
0: 512x640 1 Drone, Done. (0.209s)
0: 512x640 1 Drone, Done. (0.191s)
0: 512x640 1 Drone, Done. (0.211s)
0: 512x640 1 Drone, Done. (0.215s)
0: 512x640 1 Drone, Done. (0.217s)
0: 512x640 1 Drone, Done. (0.216s)
0: 512x640 Done. (0.211s)
0: 512x640 Done. (0.215s)
0: 512x640 Done. (0.208s)
0: 512x640 Done. (0.209s)
0: 512x640 Done. (0.212s)
0: 512x640 Done. (0.214s)
0: 512x640 Done. (0.208s)
0

hi2
hii


SystemExit: 0

C:\Users\OpenLap\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3452: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [1]:
if __name__ == '__main__' :
    app = QApplication(sys.argv) 
    demo = Appdemo()
    demo.show()
    try : 
        sys.exit(app.exec_())
    except SystemExit : 
        print("Closing Window")

NameError: name 'QApplication' is not defined

In [4]:
print(response.content)

b'{"resultCode":0}'
